In [1]:
import gymnasium as gym
import torch

In [2]:
env = gym.make('CartPole-v0')

/Users/arghasreebanerjee/opt/anaconda3/envs/XAI_week1/lib/python3.10/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [8]:
n_state = env.observation_space.shape[0] 
print(f'The number of elements to represent a state: {n_state}')
n_action = env.action_space.n
print(f'The number of actions that the agent can take: {n_action}')

# So the agent can take any of the 2 actions from any of the 4 states.
# Each of those transistions will have a weight 
# Total number of such weights = 8 (1 state has 2 weights, so 4 states have 8 weights)

The number of elements to represent a state: 4
The number of actions that the agent can take: 2


### Each component of the state contributes differently to a certain action. For eg, action 1 = w1\*component1 + w2*component2 \* ....

In [48]:
def run_episode(env, weight): 
    state = env.reset()[0]  # initial state
    total_reward = 0
    is_done = False
    while not is_done:
        state = torch.from_numpy(state).float()
        multiply = torch.matmul(state, weight)
        # print(f'State ({state.size()}) X Weight ({weight.size()}) = {multiply.size()}')
        # print('Choosing action based on the weights of invidual actions ... ', multiply,'\n')
        action = torch.argmax(multiply) # previous line 
        state, reward, is_done, _, info = env.step(action.item()) # Instead of sampling the action from a uniform distribution directly, we assign weights and then sample in previous line
        total_reward += reward
    return total_reward
        

What is an episode? 

- The agent plays the game till the game is over. 
- The agent plays with the given weight distribution among actions.

In [22]:
weight = torch.rand(n_state, n_action)
run_episode(env, weight)

State (torch.Size([4])) X Weight (torch.Size([4, 2])) = torch.Size([2])
Choosing action based on the weights of invidual actions ...  tensor([-0.0154, -0.0543]) 

State (torch.Size([4])) X Weight (torch.Size([4, 2])) = torch.Size([2])
Choosing action based on the weights of invidual actions ...  tensor([ 0.1079, -0.0492]) 

State (torch.Size([4])) X Weight (torch.Size([4, 2])) = torch.Size([2])
Choosing action based on the weights of invidual actions ...  tensor([ 0.2279, -0.0441]) 

State (torch.Size([4])) X Weight (torch.Size([4, 2])) = torch.Size([2])
Choosing action based on the weights of invidual actions ...  tensor([ 0.3457, -0.0380]) 

State (torch.Size([4])) X Weight (torch.Size([4, 2])) = torch.Size([2])
Choosing action based on the weights of invidual actions ...  tensor([ 0.4622, -0.0297]) 

State (torch.Size([4])) X Weight (torch.Size([4, 2])) = torch.Size([2])
Choosing action based on the weights of invidual actions ...  tensor([ 0.5784, -0.0182]) 

State (torch.Size([4])

10.0

In [40]:
# Learnings in different episodes (different weights of each component of the state to action pairs):
n = 100; total_reward=0; best_reward = 0
for episode in range(n):
    weight = torch.rand(n_state, n_action)
    reward = run_episode(env, weight)
    total_reward+= reward
    print(f'Episode {episode+1}, Rewards : {reward}')
    if reward > best_reward:
        print("\nBETTER WEIGHTS FOUND !!!\n")
        best_reward = reward
        best_weights = weight

print(f'Avergae reward: {total_reward/n}')

Episode 1, Rewards : 12.0

BETTER WEIGHTS FOUND !!!

Episode 2, Rewards : 10.0
Episode 3, Rewards : 10.0
Episode 4, Rewards : 10.0
Episode 5, Rewards : 11.0
Episode 6, Rewards : 9.0
Episode 7, Rewards : 15.0

BETTER WEIGHTS FOUND !!!

Episode 8, Rewards : 196.0

BETTER WEIGHTS FOUND !!!

Episode 9, Rewards : 50.0
Episode 10, Rewards : 10.0
Episode 11, Rewards : 9.0
Episode 12, Rewards : 40.0
Episode 13, Rewards : 11.0
Episode 14, Rewards : 9.0
Episode 15, Rewards : 10.0
Episode 16, Rewards : 177.0
Episode 17, Rewards : 158.0
Episode 18, Rewards : 289.0

BETTER WEIGHTS FOUND !!!

Episode 19, Rewards : 9.0
Episode 20, Rewards : 65.0
Episode 21, Rewards : 23.0
Episode 22, Rewards : 10.0
Episode 23, Rewards : 43.0
Episode 24, Rewards : 9.0
Episode 25, Rewards : 72.0
Episode 26, Rewards : 33.0
Episode 27, Rewards : 9.0
Episode 28, Rewards : 9.0
Episode 29, Rewards : 83.0
Episode 30, Rewards : 10.0
Episode 31, Rewards : 9.0
Episode 32, Rewards : 9.0
Episode 33, Rewards : 456.0

BETTER WEIGHT

##  *best_weights* has the weights for which the reward was maximum

In [49]:
n = 100; total_reward=0
for episode in range(n):
    reward = run_episode(env, best_weights)
    total_reward+= reward
print(f'Avergae reward: {total_reward/n}')

Avergae reward: 293.16


In [50]:
env.reset()

(array([0.02952401, 0.03949724, 0.00313893, 0.00549393], dtype=float32), {})

In [51]:
env.reset()

(array([ 0.03574031,  0.04337698, -0.04148225, -0.03843143], dtype=float32),
 {})

### Why the 'reward' is not same everytime if the weights are same?

Given a constant weight, because the starting state is random, the actions can change, and therefore the reward can change. But if the policy is good (the weight for each component), a good weight policy will result in better rewards.